In [2]:
import pyrfume
import pandas as pd
from pprint import pprint
from IPython.display import Image
from predicting_odor_from_molecular_structure import *
from skmultilearn.model_selection import iterative_train_test_split

In [3]:
df = goodscents()
df

,TGSC ID,CID,Concentration %,Solvent,MolecularWeight,IsomericSMILES,IUPACName,name,Labels
0,1000111,7476,100.0,NaN,150.17,CC(=O)C1=CC=C(C=C1)OC,1-(4-methoxyphenyl)ethanone,4'-methoxyacetophenone,"[sweet, vanilla, cherry maraschino cherry, pow..."
1,1031871,7478,10.0,dipropylene glycol,152.15,COC1=CC=C(C=C1)C(=O)O,4-methoxybenzoic acid,4-methoxybenzoic acid,"[phenolic, animal, fecal, medicinal]"
2,1009281,7501,0.1,triacetin,104.15,C=CC1=CC=CC=C1,styrene,styrene,"[sweet, plastic, floral, balsamic]"
3,1001651,244,100.0,NaN,108.14,C1=CC=C(C=C1)CO,phenylmethanol,benzyl alcohol,"[sweet, floral, rose, fruity, phenolic, balsam..."
4,1001491,240,10.0,dipropylene glycol,106.12,C1=CC=C(C=C1)C=O,benzaldehyde,benzaldehyde,"[sweet, cherry, cherry maraschino cherry, nutt..."
...,...,...,...,...,...,...,...,...,...
4621,1020461,11564539,NaN,NaN,170.29,CCCCCCCC(=C)C(C)O,3-methylidenedecan-2-ol,3-methylidenedecan-2-ol,"[flowers, orange]"
4622,1027261,46195361,1.0,dipropylene glycol,146.25,CCC1OCCC(S1)C,"2-ethyl-4-methyl-1,3-oxathiane",schembl2678282,"[vegetable, floral, fruity, apricot, green, tr..."
4623,1028151,46195364,0.1,propylene glycol,174.31,CCCCC1OCCC(S1)C,"2-butyl-4-methyl-1,3-oxathiane",schembl2676606,"[spicy, leafy, peppery, banana, fruity, oily, ..."
4624,1554361,88571201,NaN,NaN,114.19,CC[C@@H](C)CCC=O,(4R)-4-methylhexanal,(r)-4-methylhexanal,"[flowery, fresh, green, warm]"


In [4]:
label_frequencies(df, labels_column='Labels').query("Count >= 30")

,Label,Count
0,fruity,1250
1,green,1070
2,sweet,1007
3,floral,910
4,woody,623
...,...,...
128,anise,31
129,orangeflower,30
130,licorice,30
131,peppery,30


In [5]:
df = reduce_df(df, minimum=30)
df.shape

(4506, 9)

### Featurization

In [8]:
df = create_all_features(df)
df

,TGSC ID,CID,Concentration %,Solvent,MolecularWeight,IsomericSMILES,IUPACName,name,Labels,mol,morgan_fp,daylight_fp
0,1000111,7476,100.0,NaN,150.17,CC(=O)C1=CC=C(C=C1)OC,1-(4-methoxyphenyl)ethanone,4'-methoxyacetophenone,"[sweet, vanilla, powdery, anisic, balsamic, ha...",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1031871,7478,10.0,dipropylene glycol,152.15,COC1=CC=C(C=C1)C(=O)O,4-methoxybenzoic acid,4-methoxybenzoic acid,"[phenolic, animal, medicinal]",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1009281,7501,0.1,triacetin,104.15,C=CC1=CC=CC=C1,styrene,styrene,"[sweet, floral, balsamic]",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1001651,244,100.0,NaN,108.14,C1=CC=C(C=C1)CO,phenylmethanol,benzyl alcohol,"[sweet, floral, rose, fruity, phenolic, balsam...",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1001491,240,10.0,dipropylene glycol,106.12,C1=CC=C(C=C1)C=O,benzaldehyde,benzaldehyde,"[sweet, cherry, nutty, fruity, powdery, almond...",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4621,1020461,11564539,NaN,NaN,170.29,CCCCCCCC(=C)C(C)O,3-methylidenedecan-2-ol,3-methylidenedecan-2-ol,[orange],<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4622,1027261,46195361,1.0,dipropylene glycol,146.25,CCC1OCCC(S1)C,"2-ethyl-4-methyl-1,3-oxathiane",schembl2678282,"[vegetable, floral, fruity, apricot, green, tr...",<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4623,1028151,46195364,0.1,propylene glycol,174.31,CCCCC1OCCC(S1)C,"2-butyl-4-methyl-1,3-oxathiane",schembl2676606,"[spicy, leafy, peppery, banana, fruity, oily, ...",<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4624,1554361,88571201,NaN,NaN,114.19,CC[C@@H](C)CCC=O,(4R)-4-methylhexanal,(r)-4-methylhexanal,"[fresh, green, warm]",<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
mordred_features = create_mordred_features(dfx)
mordred_features

100%|██████████| 4506/4506 [00:48<00:00, 93.35it/s] 


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,7.956514,7.451864,0,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,8.906935,40.567492,150.068080,7.146099,162,13,50.0,55.0,4.583333,2.611111
1,7.956514,7.451864,1,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,8.906935,40.567492,152.047344,8.002492,162,13,50.0,55.0,4.583333,2.611111
2,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,104.062600,6.503913,64,7,34.0,36.0,2.611111,2.000000
3,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,108.057515,6.753595,64,7,34.0,36.0,2.611111,2.000000
4,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,106.041865,7.574419,64,7,34.0,36.0,2.611111,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,8.065904,7.893227,0,0,13.812926,2.092177,4.184353,13.812926,1.151077,3.306818,...,8.302266,40.156271,170.167065,5.004914,256,11,46.0,46.0,5.722222,3.027778
4622,6.473351,6.191587,0,0,11.142106,2.200122,4.400244,11.142106,1.238012,3.089776,...,8.606851,37.313444,146.076536,6.351154,88,9,40.0,43.0,3.472222,2.166667
4623,7.887564,7.276324,0,0,13.649148,2.212234,4.424468,13.649148,1.240832,3.278334,...,8.711608,40.046626,174.107836,6.003718,170,11,48.0,51.0,3.972222,2.666667
4624,5.059137,5.436502,0,0,9.40926,1.989044,3.978088,9.40926,1.176158,2.88852,...,7.623153,33.372552,114.104465,5.186567,76,6,28.0,27.0,4.111111,2.166667


In [16]:
mordred_features

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,7.956514,7.451864,0,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,8.906935,40.567492,150.068080,7.146099,162,13,50.0,55.0,4.583333,2.611111
1,7.956514,7.451864,1,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,8.906935,40.567492,152.047344,8.002492,162,13,50.0,55.0,4.583333,2.611111
2,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,104.062600,6.503913,64,7,34.0,36.0,2.611111,2.000000
3,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,108.057515,6.753595,64,7,34.0,36.0,2.611111,2.000000
4,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,106.041865,7.574419,64,7,34.0,36.0,2.611111,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,8.065904,7.893227,0,0,13.812926,2.092177,4.184353,13.812926,1.151077,3.306818,...,8.302266,40.156271,170.167065,5.004914,256,11,46.0,46.0,5.722222,3.027778
4622,6.473351,6.191587,0,0,11.142106,2.200122,4.400244,11.142106,1.238012,3.089776,...,8.606851,37.313444,146.076536,6.351154,88,9,40.0,43.0,3.472222,2.166667
4623,7.887564,7.276324,0,0,13.649148,2.212234,4.424468,13.649148,1.240832,3.278334,...,8.711608,40.046626,174.107836,6.003718,170,11,48.0,51.0,3.972222,2.666667
4624,5.059137,5.436502,0,0,9.40926,1.989044,3.978088,9.40926,1.176158,2.88852,...,7.623153,33.372552,114.104465,5.186567,76,6,28.0,27.0,4.111111,2.166667


In [17]:
df

,TGSC ID,CID,Concentration %,Solvent,MolecularWeight,IsomericSMILES,IUPACName,name,Labels,mol,morgan_fp,daylight_fp
0,1000111,7476,100.0,NaN,150.17,CC(=O)C1=CC=C(C=C1)OC,1-(4-methoxyphenyl)ethanone,4'-methoxyacetophenone,"[sweet, vanilla, powdery, anisic, balsamic, ha...",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1031871,7478,10.0,dipropylene glycol,152.15,COC1=CC=C(C=C1)C(=O)O,4-methoxybenzoic acid,4-methoxybenzoic acid,"[phenolic, animal, medicinal]",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1009281,7501,0.1,triacetin,104.15,C=CC1=CC=CC=C1,styrene,styrene,"[sweet, floral, balsamic]",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1001651,244,100.0,NaN,108.14,C1=CC=C(C=C1)CO,phenylmethanol,benzyl alcohol,"[sweet, floral, rose, fruity, phenolic, balsam...",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1001491,240,10.0,dipropylene glycol,106.12,C1=CC=C(C=C1)C=O,benzaldehyde,benzaldehyde,"[sweet, cherry, nutty, fruity, powdery, almond...",<rdkit.Chem.rdchem.Mol object at 0x00000233936...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4621,1020461,11564539,NaN,NaN,170.29,CCCCCCCC(=C)C(C)O,3-methylidenedecan-2-ol,3-methylidenedecan-2-ol,[orange],<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4622,1027261,46195361,1.0,dipropylene glycol,146.25,CCC1OCCC(S1)C,"2-ethyl-4-methyl-1,3-oxathiane",schembl2678282,"[vegetable, floral, fruity, apricot, green, tr...",<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4623,1028151,46195364,0.1,propylene glycol,174.31,CCCCC1OCCC(S1)C,"2-butyl-4-methyl-1,3-oxathiane",schembl2676606,"[spicy, leafy, peppery, banana, fruity, oily, ...",<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4624,1554361,88571201,NaN,NaN,114.19,CC[C@@H](C)CCC=O,(4R)-4-methylhexanal,(r)-4-methylhexanal,"[fresh, green, warm]",<rdkit.Chem.rdchem.Mol object at 0x00000233B77...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [24]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from skmultilearn.model_selection import iterative_train_test_split

# Przygotuj słownik cech
feature_sets = {
    'Morgan': np.array(df['morgan_fp'].tolist()),
    'Daylight': np.array(df['daylight_fp'].tolist()),
    'Mordred': mordred_features.values,
}

per_label_results = {}
inference_examples = {}

for name, X in feature_sets.items():
    print(f"\n=== {name} ===")
    # Dodaj kolumnę z indeksami do X
    indices = np.arange(len(df))
    X_with_idx = np.concatenate([X, indices.reshape(-1, 1)], axis=1)
    
    # Split z zachowaniem indeksów
    X_train, y_train, X_test, y_test = iterative_train_test_split(X_with_idx, Y, test_size=0.2)
    train_indices = X_train[:, -1].astype(int)
    test_indices = X_test[:, -1].astype(int)
    
    # Usuń indeksy z cech przed trenowaniem
    X_train_ = X_train[:, :-1]
    X_test_  = X_test[:, :-1]
    
    # Model
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train_, y_train)
    y_pred = model.predict(X_test_)
    if hasattr(y_pred, "toarray"):
        y_pred = y_pred.toarray()
    
    # Per-label metrics
    precision = precision_score(y_test, y_pred, average=None, zero_division=0)
    recall = recall_score(y_test, y_pred, average=None, zero_division=0)
    f1 = f1_score(y_test, y_pred, average=None, zero_division=0)
    results = pd.DataFrame({
        'label': mlb.classes_,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })
    per_label_results[name] = results

    # Inferencja: wybrane próbki z testu (na oryginalnym df)
    sample_indices = np.random.choice(len(test_indices), size=5, replace=False)
    samples = []
    for idx in sample_indices:
        original_idx = test_indices[idx]
        molecule_name = df.iloc[original_idx]['name']
        true_labels = mlb.inverse_transform(y_test[idx].reshape(1, -1))[0]
        pred_labels = mlb.inverse_transform(y_pred[idx].reshape(1, -1))[0]
        samples.append({
            'idx': int(original_idx),
            'name': molecule_name,
            'true': true_labels,
            'pred': pred_labels
        })
    inference_examples[name] = samples

    # Możesz też łatwo uzyskać df_train/df_test dla tej cechy:
    df_train = df.iloc[train_indices]
    df_test = df.iloc[test_indices]
    # ...i np. zapisać je lub użyć do dalszej analizy

# Przykład podglądu inferencji:
for name, samples in inference_examples.items():
    print(f"\n{name} - przykładowe predykcje:")
    for s in samples:
        print(f"Idx: {s['idx']} | Nazwa: {s['name']}")
        print(f"  Prawdziwe etykiety: {s['true']}")
        print(f"  Predykowane etykiety: {s['pred']}")
        print("-" * 40)




=== Morgan ===

=== Daylight ===

=== Mordred ===

Morgan - przykładowe predykcje:
Idx: 1973 | Nazwa: (2-methoxyethyl)benzene
  Prawdziwe etykiety: ('green', 'floral', 'rose', 'metallic', 'jasmin')
  Predykowane etykiety: ('green', 'floral')
----------------------------------------
Idx: 679 | Nazwa: oxomagnesium
  Prawdziwe etykiety: ('odorless',)
  Predykowane etykiety: ('odorless',)
----------------------------------------
Idx: 2224 | Nazwa: 2,3-butanedione
  Prawdziwe etykiety: ('sweet', 'creamy', 'caramellic', 'pungent', 'buttery')
  Predykowane etykiety: ()
----------------------------------------
Idx: 20 | Nazwa: 1h-pyrrole-2-carbaldehyde
  Prawdziwe etykiety: ('musty', 'coffee')
  Predykowane etykiety: ()
----------------------------------------
Idx: 1389 | Nazwa: methyl propyl disulfide
  Prawdziwe etykiety: ('sulfurous', 'onion', 'garlic', 'alliaceous')
  Predykowane etykiety: ('sulfurous', 'metallic', 'onion', 'alliaceous')
----------------------------------------

Daylight 

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer

classes = label_frequencies(df, labels_column='Labels').Label.to_list()
mlb = MultiLabelBinarizer(classes=classes)
Y = mlb.fit_transform(df['Labels'])

mlb, Y.shape

(MultiLabelBinarizer(classes=['fruity', 'green', 'sweet', 'floral', 'woody',
                              'herbal', 'fatty', 'fresh', 'waxy', 'spicy',
                              'citrus', 'rose', 'sulfurous', 'earthy', 'nutty',
                              'tropical', 'odorless', 'balsamic', 'oily',
                              'minty', 'vegetable', 'powdery', 'musty', 'meaty',
                              'creamy', 'pineapple', 'apple', 'aldehydic',
                              'roasted', 'phenolic', ...]),
 (4506, 133))

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import ClassifierChain

In [15]:
from sklearn.metrics import f1_score, precision_score, recall_score
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np
import pandas as pd

feature_sets = {
    "Morgan": np.array(df['morgan_fp'].tolist()),
    "Daylight": np.array(df['daylight_fp'].tolist()),
    "Mordred": mordred_features.values,
}

results_tables = {}
inference_samples = {}

for name, X in feature_sets.items():
    print(f"=== {name} ===")
    # Split (zwraca 4 tablice: X_train, y_train, X_test, y_test)
    X_train, y_train, X_test, y_test = iterative_train_test_split(X, Y, test_size=0.2)
    
    # Model
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if hasattr(y_pred, "toarray"):
        y_pred = y_pred.toarray()
    
    # Per-label metrics
    precision = precision_score(y_test, y_pred, average=None)
    recall = recall_score(y_test, y_pred, average=None)
    f1 = f1_score(y_test, y_pred, average=None)
    results = pd.DataFrame({
        'label': mlb.classes_,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })
    results_tables[name] = results
    
    # Inferencja - wybrane próbki
    sample_indices = np.random.choice(X_test.shape[0], size=5, replace=False)
    sample_info = []
    for idx in sample_indices:
        # Zakładam, że masz indeksy oryginalnych wierszy dla X_test
        # Jeśli nie, możesz je uzyskać np. przez przekazanie indeksów do iterative_train_test_split
        # Tu uproszczenie:
        original_idx = idx  # <- zamień na prawidłowy indeks jeśli masz mapping!
        molecule_name = df.iloc[original_idx]['name']
        true_labels = mlb.inverse_transform(y_test[idx].reshape(1, -1))[0]
        pred_labels = mlb.inverse_transform(y_pred[idx].reshape(1, -1))[0]
        sample_info.append({
            "idx": original_idx,
            "name": molecule_name,
            "true": true_labels,
            "pred": pred_labels
        })
    inference_samples[name] = sample_info


=== Morgan ===


c:\Users\mikoo\miniconda3\envs\primo\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


=== Daylight ===


c:\Users\mikoo\miniconda3\envs\primo\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


=== Mordred ===


c:\Users\mikoo\miniconda3\envs\primo\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.metrics import f1_score, precision_score, recall_score
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# Przygotuj MultiLabelBinarizer i macierz etykiet
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['Labels'])

# Przygotuj cechy
feature_sets = {
    'Morgan': np.array(df['morgan_fp'].tolist()),
    'Daylight': np.array(df['daylight_fp'].tolist()),
    'Mordred': mordred_features.values,
}


In [27]:
indices = np.arange(len(df))
# Dodaj indeks jako ostatnią kolumnę do cech (np. Mordred)
# Wybierz dowolny feature set do splitowania, bo indeksy są te same dla wszystkich
X_with_idx = np.concatenate([feature_sets['Mordred'], indices.reshape(-1, 1)], axis=1)
X_train, y_train, X_test, y_test = iterative_train_test_split(X_with_idx, Y, test_size=0.2)
train_indices = X_train[:, -1].astype(int)
test_indices = X_test[:, -1].astype(int)

# Funkcja do wyciągania podzbioru cech po indeksach
def get_split(X, train_idx, test_idx):
    return X[train_idx], X[test_idx]


In [ ]:
get_split()